In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

# Read the data

In [3]:
dfXtrain = pd.read_csv('preprocessed_csv/train_more.csv', index_col='id')
dfXtest = pd.read_csv('preprocessed_csv/test_more.csv', index_col='id')
dfYtrain = pd.read_csv('preprocessed_csv/y_train_more.csv', header=None, names=['INDEX', 'P_TARGET_FLAG'])

In [4]:
x_train = np.array(dfXtrain)
x_test = np.array(dfXtest)

y_train = np.array(dfYtrain['P_TARGET_FLAG'])

# Save routines

In [5]:
dfYtest = pd.DataFrame({'INDEX': dfXtest.index, 'P_TARGET_FLAG': np.zeros(x_test.shape[0])})
dfYtest.head()

,INDEX,P_TARGET_FLAG
0,3,0.0
1,9,0.0
2,10,0.0
3,18,0.0
4,21,0.0


In [6]:
def save_to_file(y, file_name):
    dfYtest['P_TARGET_FLAG'] = y
    dfYtest.to_csv('results/{}'.format(file_name), index=False)

# Train RF

In [7]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split

In [8]:
%%time

kwargs = {'n_estimators': 5000, 'class_weight': None, 'max_features': 'auto',
          'criterion': 'entropy', 'n_jobs': -1}

proba = np.zeros(x_test.shape[0])

for random_state in tqdm_notebook(range(1, 21), desc='random_state'):
    rfc = RFC(random_state=random_state, **kwargs)
    rfc.fit(x_train, y_train)
    proba += rfc.predict_proba(x_test)[:, 1]
    del rfc
    gc.collect()


CPU times: user 57min 22s, sys: 21.3 s, total: 57min 44s
Wall time: 16min 17s


In [9]:
proba /= 20

In [10]:
save_to_file(proba, 'more_100_000_cycle.csv')